In [137]:
import numpy as np
import pandas as pd
import math
import operator

In [138]:
def norma(a):
  a = np.array(a)
  at = a.T
  return math.sqrt(np.dot(a,at)) 

In [139]:
def normalizar(a):
  a = np.array(a)
  return a/norma(a)

In [140]:
def dist(a,b):
  a = np.array(a)
  b = np.array(b)
  return norma(b-a)

In [141]:
def projecao(a,b):
  a = np.array(a)
  b = np.array(b)
  return np.dot(a,b)/np.dot(a,a)*a

In [142]:
def cos(a,b):
    p = projecao(a,b)
    return norma(p)/norma(b)

In [143]:
def avarage_vector(a):
    a = np.array(a)
    return np.mean(a,axis=0)

In [144]:
a = np.array([[1,2,3],[2,3,4],[7,8,9]])
avarage_vector(a)

array([3.33333333, 4.33333333, 5.33333333])

In [145]:
def tratar_df(df): # trata o dataframe, excluindo colunas inuteis.
    df = df.loc[:, 'pace':'goalkeeping_reflexes']
    df = df.dropna()
    df = df.dropna(axis=0)
    #df = df.drop(['player_positions','long_name', 'wage_eur', 'dob', 'height_cm', 'weight_kg', 'club_team_id'], axis=1)
    #df = df.drop(df.loc[:, 'club_jersey_number':'player_traits'], axis=1)
    df = df.to_numpy()
    return df

In [146]:
def gerar_dfs(df, pos):
    dfs = []
    for i in pos: 
        dfs.append(avarage_vector(tratar_df(df[df['club_position'] == i])))
    return dfs

In [147]:
def predict(dict, pos_nomes, player):
    coss = []
    for p in pos_nomes:
        coss.append(cos(player[0], dict[p]))
    pos = {pos_nomes[i]:coss[i] for i in range(len(pos_nomes))}
    return max(pos, key=pos.get)


In [148]:
def team_over(df, team, ata, mei, deff):
    over = [0,0,0,0]
    over_ = [0,0,0]
    df = df[df['club_name']==team]
    df = df[df['club_position'] != 'RES']
    for _,player in df.iterrows():
        if any(item in player['player_positions'] for item in ata):
            over[0] = over[0] + player['overall']
            over_[0] = over_[0] + 1
        if any(item in player['player_positions'] for item in mei):
            over[1] = over[1] + player['overall']
            over_[1] = over_[1] + 1
        if any(item in player['player_positions'] for item in deff):
            over[2] = over[2] + player['overall']
            over_[2] = over_[2] + 1
    over[0] = over[0]/over_[0]
    over[1] = over[1]/over_[1]
    over[2] = over[2]/over_[2]
    over[3] = (over[0]+over[1]+over[2])/3
    return over
    


In [154]:
from ast import operator


def player_dist(df, player):
    player_df = df[df['short_name'] == player]
    dfs = df.drop(df[df['short_name'] == player].index)	# remove o player do df
    dfs = tratar_df(dfs)
    player_df = tratar_df(player_df)
    dists = []
    for i in dfs:
        dists.append(cos(i, player_df[0]))
    dists = dict(zip(df['short_name'], dists))  
    dists = dict(sorted(dists.items(), key = lambda x: x[1], reverse = True))
    return dists

In [150]:
ata_pos = ["ST", "LF", "CF", "RF", "LW", "RW"] # posicoes do atacante
mei_pos = ["CAM", "CM", "CDM", "LM", "RM"] # posicoes do meio
def_pos = ["CB", "LB", "RB", "LWB", "RWB", "GK"] # posicoes do defesa
pos_nomes = ata_pos + mei_pos + def_pos
df = pd.read_csv('players_22.csv')
pos_df = gerar_dfs(df, pos_nomes)
pos_dict = dict(zip(pos_nomes, pos_df))

C:\Users\bieli\AppData\Local\Temp/ipykernel_24920/1556863144.py:5: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('players_22.csv')
c:\Users\bieli\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\bieli\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [151]:
team_over(df, 'Tottenham Hotspur', ata_pos, mei_pos, def_pos)

[79.42857142857143, 79.83333333333333, 79.54545454545455, 79.60245310245311]

In [152]:
pl_teams = df[df['league_name'] == 'English Premier League']
pl_over = []
for team in pl_teams['club_name'].unique():
    pl_over.append(team_over(df, team, ata_pos, mei_pos, def_pos))
pl_over = np.array(pl_over).mean()
print(pl_over)
    

76.71144253894253


In [156]:
player_dist(df, "L. Messi")

{'Falcao': 0.9968644830003048,
 'Nani': 0.9964363023179037,
 'S. Agüero': 0.9961649132830807,
 'F. Kostić': 0.9959938700604799,
 'J. Vardy': 0.9958846082480651,
 'P. Ševčík': 0.9954115261285879,
 'M. Taremi': 0.9953387597122092,
 'J. Sancho': 0.9951960746979559,
 'N. Viergever': 0.9950974726115664,
 'Cristiano Ronaldo': 0.9949252546481515,
 'P. Aquino': 0.9947749493121386,
 'Jacson Zonta': 0.9947158771397135,
 'F. Onyeka': 0.9947023559528148,
 'Gustavo Assunção': 0.994693076943649,
 'G. Clichy': 0.9944995353529247,
 'R. Christie': 0.994478195522779,
 'O. Rivero': 0.9944553160964794,
 'P. Gallese': 0.9944429103538376,
 'J. Matip': 0.9943948719631979,
 'H. Yamaguchi': 0.9942619744921253,
 'J. Blaswich': 0.9942615034995302,
 'R. Leipertz': 0.9942093621294487,
 'Vicemte Tófoli': 0.9942051967499084,
 'P. van Aanholt': 0.9941633997700281,
 'P. Gollini': 0.9941111105167484,
 'R. Marin': 0.9940013054938673,
 'S. Larsson': 0.9939663272362591,
 'S. Gnabry': 0.9938803370804279,
 'Carraça': 0.9938